In [1]:
import os
import pandas as pd
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import cv2 as cv
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
import warnings
import shutil
from tqdm import tqdm
warnings.filterwarnings('ignore')
%load_ext autotime

time: 0 ns (started: 2023-03-14 21:59:23 +05:30)


# IMG  DIR

In [3]:
# len(os.listdir('orginalImg/'))

time: 0 ns (started: 2023-03-14 21:43:04 +05:30)


In [4]:
files=os.listdir('raw_img/')
files[:10]

['img017-001.png',
 'img017-002.png',
 'img017-003.png',
 'img017-004.png',
 'img017-005.png',
 'img017-006.png',
 'img017-007.png',
 'img017-008.png',
 'img017-009.png',
 'img017-010.png']

time: 15 ms (started: 2023-03-14 21:59:50 +05:30)


In [16]:
data_set=pd.read_csv('Characters.csv')
data_set.head()

,img,class
0,Img/img017-001.png,17
1,Img/img017-002.png,17
2,Img/img017-003.png,17
3,Img/img017-004.png,17
4,Img/img017-005.png,17


time: 16 ms (started: 2023-03-14 22:10:10 +05:30)


In [17]:
data_set.shape

(425, 2)

time: 15 ms (started: 2023-03-14 22:10:10 +05:30)


# Resize Test

In [9]:
img=cv.imread(f'raw_img/{files[4]}')
print(files[4])
cv.imshow('sample',img)
cv.waitKey(0)

img648-005.png


-1

time: 1.78 s (started: 2023-03-14 20:58:46 +05:30)


In [6]:
tranform_img=resize(img,(150,150,3))
cv.imshow('tranformed',tranform_img)
cv.waitKey(0)

-1

time: 1.84 s (started: 2023-03-14 20:18:21 +05:30)


# Image Tranformation

In [5]:
# os.makedirs('img')
for i in tqdm(range(len(files))):
    img=cv.imread(f'raw_img/{files[i]}')
    tranform_img=resize(img,(150,150,3))
    path=f'img/{files[i]}'
    cv.imwrite(path, tranform_img*255)

100%|████████████████████████████████████████████████████████████████████████████████| 425/425 [00:53<00:00,  7.97it/s]

time: 53.3 s (started: 2023-03-14 22:00:01 +05:30)


# Feature Extraction

In [18]:
feature=[]
target=[]
for i in tqdm(range(len(os.listdir('Img/')))):
    img_path=(data_set['img'].iloc[i])

    img=imread(img_path) #tranform image
    flatten_img=img.flatten()

    feature.append(flatten_img)
    target.append(data_set['class'].iloc[i])


feature=np.array(feature)
target=np.array(target)

100%|███████████████████████████████████████████████████████████████████████████████| 425/425 [00:02<00:00, 200.01it/s]

time: 2.16 s (started: 2023-03-14 22:10:16 +05:30)


# Splitting with 30% for testing

In [13]:
X_train,X_test,y_train,y_test=train_test_split(feature,target,test_size=.2,random_state=21)

time: 15 ms (started: 2023-03-14 20:59:24 +05:30)


In [14]:
X_train.shape,X_test.shape

((200, 67500), (50, 67500))

time: 0 ns (started: 2023-03-14 20:59:24 +05:30)


In [22]:
model1=SVC()
model1.fit(feature,target)
# model1.score(feature,target)*100,model1.score(X_test,y_test)*100

SVC()

time: 20 s (started: 2023-03-14 21:02:31 +05:30)


In [28]:
model2=RandomForestClassifier()
model2.fit(feature,target)

RandomForestClassifier()

time: 1.27 s (started: 2023-03-14 21:05:24 +05:30)


In [19]:
model3=LogisticRegression()
model3.fit(feature,target)

LogisticRegression()

time: 32.5 s (started: 2023-03-14 22:10:24 +05:30)


<!-- # # svc=SVC()

# # params={
# #     'C':[1,2,3,4,5],
# #     'kernel':['poly','rbf','sigmoid'],
# #     'gamma':['auto','scale']
# # }
# # GSsvm=GridSearchCV(svc,param_grid=params,n_jobs=1)
# # GSsvm.fit(X_train,y_train)
# # print(GSsvm.best_params_)
# # print(GSsvm.best_score_) -->

In [37]:
svc=SVC()

params={
    'C':[1,2,3,4,5],
    'kernel':['poly','rbf','sigmoid'],
    'gamma':['auto','scale']
}
GSsvm=GridSearchCV(svc,param_grid=params,n_jobs=1)
GSsvm.fit(X_train,y_train)
print(GSsvm.best_params_)
print(GSsvm.best_score_)

time: 0 ns (started: 2023-03-14 20:45:51 +05:30)


In [ ]:
GSsvm.score(feature,target)*100,GSsvm.score(X_test,y_test)*100

# Predicting

In [20]:
def tranform_image(img_path):
    feature=[]
    img=imread(f'{img_path}.png')
    tranform_img=resize(img,(150,150,3))
    flatten_img=tranform_img.flatten()
    feature.append(flatten_img)
    return np.array(feature)

time: 0 ns (started: 2023-03-14 22:10:57 +05:30)


In [22]:
img_name=f'test_img/{input()}'
# print(model1.predict(tranform_image(img_name)))
# print(model2.predict(tranform_image(img_name)))
print(model3.predict(tranform_image(img_name)))

 img018-002


[18]
time: 3.55 s (started: 2023-03-14 22:11:54 +05:30)


# Dumping Model for future use

In [23]:
import pickle

pickle.dump(model3,open('KCR.pkl','wb'))

time: 31 ms (started: 2023-03-14 22:12:05 +05:30)
